In [1]:
import numpy as np
import pandas as pd

# Demographic

In [2]:
member = pd.read_csv('/home/liutianc/emr-data/member_detail.csv')
member['birth'] = member.Yrdob.astype(int)

member = member[(2010 - member['birth']) >= 18]
member = member.drop_duplicates()

In [4]:
member

,Patid,Gdr_Cd,Yrdob,birth
0,560499200160862,M,1962,1962
2,560499200302981,M,1965,1965
4,560499200668649,F,1956,1956
5,560499200782112,M,1973,1973
12,560499201057366,M,1981,1981
...,...,...,...,...
7302801,560499899999952,F,1978,1978
7302804,560499899999970,M,1959,1959
7302806,560499899999980,F,1969,1969
7302810,560499899999984,F,1987,1987


# IndexVTE

In [5]:
vte = pd.read_csv('/home/liutianc/emr-vte/diag_vte.csv')
vte['date'] = pd.to_datetime(vte.Fst_Dt, format='%Y-%m-%d')

In [7]:
final_user = []
for year in range(2011, 2019):
    baseline = vte[vte.date < np.datetime64(f'{year}-01-01')]
    baseline = baseline.groupby('Patid').agg({'date': max}).reset_index()
    possible = vte[vte.date >= np.datetime64(f'{year}-01-01')]
    possible = possible.groupby('Patid').agg({'date': min}).reset_index()
    
    vte_data =  pd.merge(possible, baseline, on='Patid', how='left')
    vte_first = vte_data[np.isnat(vte_data.date_y)]
    final_user.append(vte_first)

    vte_old = vte_data[~np.isnat(vte_data.date_y)]
    vte_old = vte_old[vte_old.date_x > vte_old.date_y + np.timedelta64(365, 'D')]
    final_user.append(vte_old)

In [8]:
final_user1 = pd.concat(final_user)
final_user1 = final_user1.groupby('Patid').agg({'date_x': min}).reset_index()
final_user1 = final_user1.rename(columns={'date_x': 'vte_date'})
final_user1.to_csv('IndexVTE.csv')

final_user1 = pd.merge(final_user1, member[['Patid']], on='Patid')
vte_user = final_user1
vte_user

,Patid,vte_date
0,560499202223440,2014-08-13
1,560499202246702,2015-05-11
2,560499202260355,2014-02-14
3,560499202269085,2014-08-26
4,560499202308710,2013-01-19
...,...,...
41570,560499899987606,2012-08-27
41571,560499899987724,2011-07-12
41572,560499899990407,2011-06-16
41573,560499899996407,2013-01-12


# Diag Cancer

In [9]:
diag_cancer = pd.read_csv('/home/liutianc/emr-vte/diag_cancer.csv')
diag_cancer['diag_cancer_date'] = pd.to_datetime(diag_cancer.Fst_Dt, format='%Y-%m-%d')
diag_cancer.head()

,Patid,Clmid,Diag,Diag_Position,Icd_Flag,Loc_cd,Fst_Dt,diag_cancer_date
0,560499800047333,JV3LL9RON,1401,1.0,9,2,2010-04-13,2010-04-13
1,560499800047333,JV9ORFRNN,1401,1.0,9,2,2010-01-12,2010-01-12
2,560499800047333,JVL83ORNN,1401,1.0,9,2,2010-07-13,2010-07-13
3,560499800047333,JVN9OFNFL,1401,1.0,9,2,2010-04-21,2010-04-21
4,560499800047333,JVNFFVF9R,1401,1.0,9,2,2010-04-20,2010-04-20


In [10]:
vte_diag_cancer = pd.merge(diag_cancer, vte_user, on='Patid', how='inner')
vte_diag_cancer = vte_diag_cancer[vte_diag_cancer.diag_cancer_date < vte_diag_cancer.vte_date]
vte_diag_cancer.to_csv('DiagCancerVTEUser.csv')

In [11]:
vte_user = vte_diag_cancer.groupby('Patid').agg({'diag_cancer_date': min, 'vte_date': min}).reset_index()
vte_user

,Patid,diag_cancer_date,vte_date
0,560499202269085,2014-04-22,2014-08-26
1,560499202308710,2010-02-02,2013-01-19
2,560499204124088,2013-05-13,2014-08-13
3,560499204544501,2011-06-28,2014-02-17
4,560499208058839,2015-03-31,2015-06-03
...,...,...,...
8401,560499899961821,2010-02-08,2015-02-24
8402,560499899973563,2010-04-08,2015-01-15
8403,560499899973907,2010-10-14,2014-02-09
8404,560499899987027,2010-11-04,2011-06-05


# Treatment Cancer

In [12]:
pharm_cancer = pd.read_csv('/home/liutianc/emr-vte/pharm_cancer.csv')
pharm_cancer['date'] = pd.to_datetime(pharm_cancer.Fill_Dt, format='%Y-%m-%d')
pharm_cancer.head()

,Patid,Brnd_Nm,Clmid,Fill_Dt,Gnrc_Nm,Quantity,Rfl_Nbr,date
0,560499843179936,EYLEA,J89J83RVLV,2011-12-27,AFLIBERCEPT,0.15,0.0,2011-12-27
1,560499802444271,EYLEA,JLJF8LRJON,2012-05-16,AFLIBERCEPT,5.00,0.0,2012-05-16
2,560499802444271,EYLEA,JL39J9FORV,2012-06-05,AFLIBERCEPT,0.10,0.0,2012-06-05
3,560499802444271,EYLEA,JLN8JVF88R,2012-06-28,AFLIBERCEPT,0.10,2.0,2012-06-28
4,560499802444271,EYLEA,JL8FRR8VOV,2012-07-27,AFLIBERCEPT,0.10,3.0,2012-07-27


In [13]:
proc_cancer = pd.read_csv('/home/liutianc/emr-vte/proc_cancer.csv')
proc_cancer['date'] = pd.to_datetime(proc_cancer.Fst_Dt, format='%Y-%m-%d')
proc_cancer.head()

,Patid,Clmid,Icd_Flag,Proc,Proc_Position,Fst_Dt,date
0,560499844334244,3J9LVJJNR3,9,J1100,2,2010-12-16,2010-12-16
1,560499887424762,3J9LRJJ9FJ,9,J1100,2,2010-06-07,2010-06-07
2,560499893889679,OVJJRRVF9N,9,J9280,2,2010-09-07,2010-09-07
3,560499893928122,OVJJNJV3NV,9,J9280,3,2010-10-12,2010-10-12
4,560499895375085,399OJ93VLF,9,J9280,2,2010-01-19,2010-01-19


In [14]:
treat_cancer = pd.concat([pharm_cancer, proc_cancer])
treat_cancer = treat_cancer.rename(columns={'date': 'treatment_cancer_date'})

vte_treat_cancer = pd.merge(treat_cancer, vte_user, on='Patid', how='inner')
vte_treat_cancer = vte_treat_cancer[vte_treat_cancer.treatment_cancer_date < vte_treat_cancer.vte_date]
vte_treat_cancer = vte_treat_cancer[vte_treat_cancer.treatment_cancer_date >= vte_treat_cancer.diag_cancer_date]
vte_treat_cancer.to_csv('TreatCancerVTEUser.csv')

In [15]:
vte_user = vte_treat_cancer.groupby('Patid').agg({'diag_cancer_date': min, 'treatment_cancer_date': min, 'vte_date': min}).reset_index()
vte_user

,Patid,diag_cancer_date,treatment_cancer_date,vte_date
0,560499208408385,2014-08-04,2014-08-26,2014-12-22
1,560499208996134,2014-02-26,2014-03-14,2014-10-24
2,560499221373256,2013-09-17,2013-10-11,2014-02-19
3,560499229070560,2014-01-27,2014-02-28,2014-12-17
4,560499273351311,2014-04-22,2015-01-20,2015-08-17
...,...,...,...,...
1443,560499899818601,2012-08-09,2012-11-30,2015-03-05
1444,560499899852407,2010-05-10,2011-06-02,2014-04-30
1445,560499899874167,2010-04-13,2010-06-21,2011-06-23
1446,560499899928784,2010-11-18,2011-03-08,2011-04-04


# AC

In [16]:
pharm_ac = pd.read_csv('/home/liutianc/emr-vte/pharm_antico.csv')
pharm_ac['ac_date'] = pd.to_datetime(pharm_ac.Fill_Dt, format='%Y-%m-%d')
pharm_ac.head()

,Patid,Brnd_Nm,Clmid,Fill_Dt,Gnrc_Nm,Quantity,Rfl_Nbr,ac_date
0,560499202308710,WARFARIN_SODIUM,J9V8RR99OL,2010-04-06,WARFARIN_SODIUM,30.0,0.0,2010-04-06
1,560499202308710,WARFARIN_SODIUM,JJLJNOONNV,2010-07-19,WARFARIN_SODIUM,90.0,0.0,2010-07-19
2,560499202308710,WARFARIN_SODIUM,JORFJ3JLLJ,2010-12-21,WARFARIN_SODIUM,90.0,1.0,2010-12-21
3,560499202308710,WARFARIN_SODIUM,JJ8JFNF39V,2010-07-05,WARFARIN_SODIUM,90.0,2.0,2010-07-05
4,560499202308710,WARFARIN_SODIUM,JOR8OV838F,2010-12-21,WARFARIN_SODIUM,90.0,0.0,2010-12-21


In [17]:
pharm_ac = pd.merge(pharm_ac, vte_user, on='Patid', how='inner')
vte_pharm_ac = pharm_ac[(pharm_ac.ac_date >= pharm_ac.vte_date) & (pharm_ac.ac_date < pharm_ac.vte_date + np.timedelta64(30, 'D'))]
vte_pharm_ac

,Patid,Brnd_Nm,Clmid,Fill_Dt,Gnrc_Nm,Quantity,Rfl_Nbr,ac_date,diag_cancer_date,treatment_cancer_date,vte_date
343,560499800697493,WARFARIN_SODIUM,O3NVV3NJRF,2014-08-04,WARFARIN_SODIUM,30.0,0.0,2014-08-04,2010-01-21,2010-02-02,2014-07-28
429,560499801379409,COUMADIN,J8OOFL3FON,2012-01-03,WARFARIN_SODIUM,90.0,4.0,2012-01-03,2010-01-21,2010-02-08,2011-12-28
750,560499801640012,WARFARIN_SODIUM,JVL89JN98F,2013-05-10,WARFARIN_SODIUM,15.0,0.0,2013-05-10,2010-08-10,2012-05-14,2013-04-25
802,560499801719909,WARFARIN_SODIUM,O8J939FFL8,2015-05-26,WARFARIN_SODIUM,30.0,0.0,2015-05-26,2011-06-28,2011-08-13,2015-05-05
861,560499801905046,WARFARIN_SODIUM,OF9JR3NFLJ,2015-01-23,WARFARIN_SODIUM,30.0,1.0,2015-01-23,2010-01-08,2011-11-01,2015-01-23
...,...,...,...,...,...,...,...,...,...,...,...
18304,560499892475780,XARELTO,O8VL9393FO,2015-09-01,RIVAROXABAN,40.0,0.0,2015-09-01,2010-01-26,2010-02-15,2015-08-31
18314,560499893222902,XARELTO,O8OF9N3833,2015-06-10,RIVAROXABAN,90.0,0.0,2015-06-10,2010-01-07,2010-02-19,2015-05-24
18333,560499893463144,XARELTO,O8FFN8NRNN,2015-07-15,RIVAROXABAN,30.0,0.0,2015-07-15,2010-11-24,2011-01-31,2015-07-13
18340,560499893463144,XARELTO,O8F3FFNJ88,2015-07-13,RIVAROXABAN,51.0,0.0,2015-07-13,2010-11-24,2011-01-31,2015-07-13


In [18]:
vte_user = vte_pharm_ac.groupby('Patid').agg({'diag_cancer_date': min, 'treatment_cancer_date': min, 'vte_date': min, 'ac_date': min}).reset_index()
vte_user

,Patid,diag_cancer_date,treatment_cancer_date,vte_date,ac_date
0,560499208408385,2014-08-04,2014-08-26,2014-12-22,2014-12-22
1,560499208996134,2014-02-26,2014-03-14,2014-10-24,2014-10-27
2,560499221373256,2013-09-17,2013-10-11,2014-02-19,2014-02-22
3,560499273351311,2014-04-22,2015-01-20,2015-08-17,2015-08-19
4,560499293183608,2014-03-17,2014-05-06,2014-11-18,2014-12-16
...,...,...,...,...,...
558,560499899528851,2011-11-08,2011-12-12,2012-01-19,2012-01-20
559,560499899597605,2010-07-22,2013-02-19,2014-08-25,2014-08-28
560,560499899655199,2010-02-03,2010-02-16,2012-11-29,2012-12-12
561,560499899852407,2010-05-10,2011-06-02,2014-04-30,2014-05-20
